# Term Similarity
## subsection of _Text Similarity and Clustering_

* Analyzing Term Similarity
    1. Hamming Distance
    2. Manhattan Distance
    3. Euclidean Distance
    4. Levenshtein Edit Distance
    5. Cosine Distance and Similarity

# Analyzing Term Similarity

In [ ]:
# character vectorization: mapping each character of term to corresponding unique number
import numpy as np

# fxn takes list of words and terms, returns corresponding character vector for words
def vectorize_terms(terms):
    terms = [term.lower() for term in terms]
    terms = [np.array(list(term)) for term in terms]
    terms = [np.array([ord(char) for char in term]) for term in terms]
    return terms

# example terms
root = 'Believe'
term1 = 'beleive'
term2 = 'bargain'
term3 = 'Elephant'

terms = [root, term1, term2, term3]
terms

In [ ]:
import pandas as pd

# perform character vectorization on strings
# view representation in dataframe

# character vectorization
term_vectors = vectorize_terms(terms)

# show vector representation
vec_df = pd.DataFrame(term_vectors, index=terms)
print(vec_df)

In [ ]:
# store above info
root_term = root
other_terms = [term1, term2, term3]

root_term_vec = vec_df[vec_df.index == root_term].dropna(axis=1).values[0]
other_term_vecs = [vec_df[vec_df.index == term].dropna(axis=1).values[0]
                      for term in other_terms]

## Hamming Distance
* distance measured between two strings under the assumption that they are of equal length
* number of positions that have different characters or symbols between two strings of equal length

* consider two terms $u$ and $v$ of length $n$; denote hamming distance as follows:

$$hd(u,v) = \sum_{i=1}^n (u_i \neq v_i)$$

* can normalize by dividing the number of mismatches by the total length of the terms

$$norm\_hd(u,v) = \frac{\sum_{i=1}^n (u_i \neq v_i)}{n}$$

In [ ]:
# function computes hamming distance between two terms and can compute normalized distance
def hamming_distance(u, v, norm=False):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    return (u != v).sum() if not norm else (u != v).mean()

# measure hamming distance between root term and other terms
# compute Hamming distance
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Hamming distance between root: {} and term: {} is {}'.format(
        root_term, term, hamming_distance(root_term_vec, term_vector, norm=False)))

In [ ]:
# compute normalized hamming distance
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Normalized Hamming distance between root: {} and term: {} is {}'.format(
        root_term, term, round(hamming_distance(root_term_vec, term_vector, norm=True),2)))

## Manhattan Distance
* instead counting the number of mismatches, we subtract the difference between each pair of characters at each position of the two strings
* the distance between two points in a grid based on a strictly horizontal or vertical path
* mathematically denoted, where u and v are term terms of length n, as the following:

$$ md(u,v) = ||u-v||_l = \sum_{i=1}^{n} |u_i - v_i|$$

* same assumption of hamming distance: two terms have equal lengths
* can also have normalized manhattan distance

$$norm\_md(u,v) = \frac{||u-v||_l}{n} = \frac{\sum_{i=1}^{n} |u_i - v_i|}{n}$$

In [ ]:
# function for calculating manhattan distance with capability of calculating normalized manhattan distance
def manhattan_distance(u, v, norm=False):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    return abs(u - v).sum() if not norm else abs(u - v).mean()

# compute Manhattan distance
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Manhattan distance between root: {} and term: {} is {}'.format(
        root_term, term, manhattan_distance(root_term_vec, term_vector, norm=False)))

In [ ]:
# compute normalized Manhattan distance
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Normalized Manhattan distance between root: {} and term: {} is {}'.format(
        root_term, term, round(manhattan_distance(root_term_vec, term_vector, norm=True), 2)))

## Euclidean Distance
* the shortest straight-line distance between two points
* the formula, where u and v are vectorized text terms with length n, is the following:

$$ed(u,v) = ||u-v||_2 = \sqrt{\sum_{i=1}^n (u_i - v_i)^2}$$

In [ ]:
# function helps us compute the Euclidean distance between two terms
def euclidean_distance(u,v):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    distance = np.sqrt(np.sum(np.square(u-v)))
    return distance

# compute Euclidean distance
for term, term_vector in zip(other_terms, other_term_vecs):
    print('Euclidean distance between root: {} and term: {} is {}'.format(
        root_term, term, round(euclidean_distance(root_term_vec, term_vector), 2)))

## Levenshtein Edit Distance
* measure the distance between two sequences of strings based on their difference, similar to the concept behind the Hamming distance
* the minimum number of edits needed in the form of additions, deletions, or subsitutions to change or convert one term to the other
* character-based subsitutions: a single character can be edited in a single operation
* length of two terms need not be equal

In [ ]:
# following pseudocode, implement levenshtein edit distance
# returns final Levenshtein Edit distance and complete edit matrix between u and v
# past terms directly in raw string not vector representations
import copy
import pandas as pd

def levenshtein_edit_distance(u, v):
    # convert to lower case
    u = u.lower()
    v = v.lower()
    # base cases
    if u == v: return 0
    elif len(u) == 0: return len(v)
    elif len(v) == 0: return len(u)
    # initialize edit distance matrix
    edit_matrix = []
    # initialize two distance matrices 
    du = [0] * (len(v) + 1)
    dv = [0] * (len(v) + 1)
    # du: the previous row of edit distances
    for i in range(len(du)):
        du[i] = i
    # dv : the current row of edit distances    
    for i in range(len(u)):
        dv[0] = i + 1
        # compute cost as per algorithm
        for j in range(len(v)):
            cost = 0 if u[i] == v[j] else 1
            dv[j + 1] = min(dv[j] + 1, du[j + 1] + 1, du[j] + cost)
        # assign dv to du for next iteration
        for j in range(len(du)):
            du[j] = dv[j]
        # copy dv to the edit matrix
        edit_matrix.append(copy.copy(dv))
    # compute the final edit distance and edit matrix    
    distance = dv[len(v)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    edit_matrix = pd.DataFrame(data=edit_matrix,
                               index=list(v),
                               columns=list(u))
    return distance, edit_matrix

In [ ]:
# compute Levenshtein Edit distance between example terms
for term in other_terms:
    edit_d, edit_m = levenshtein_edit_distance(root_term, term)
    print('Computing distance between root: {} and term: {}'.format(root_term, term))
    print('Levenshtein edit distance is {}'.format(edit_d))
    print('The complete edit distance matrix is depicted below')
    print(edit_m)
    print('-'*30)

## Cosine Distance and Similarity
* have two terms represented in vectorized forms, cosine similarity gives us the measure of the cosine of the angle between them when they're represented as non-zero positive vectors in an inner product space
* term vectors with similar orientation have scores closer to 1 (cos(0))
* term vectors with a similarity score close to 0 (cos(90)) indicate unrelated terms
* mathematical definition: let u and v be two term vectors. Then,
$$u \cdot v = ||u|| ||v|| cos(\theta)$$
* can derive cosine similarity from formula:
$$cs(u,v) = cos(\theta) = \frac{u\cdot v}{||u|| ||v||} = \frac{\sum_{i=1}^n u_i v_i}{\sqrt{\sum_{i=1}^n u_i^2}\sqrt{\sum_{i=1}^n v_i^2}}$$
* use bag of characters vectorization to build these term vectors and $n$ is the number of unique characters across the terms under analysis
* bag of characters vectorization is similar to bag of words models except here we compute frequency of each character in word
* sequence or word orders are not taken into account here

In [ ]:
from scipy.stats import itemfreq

# take a list of words or terms and extract unique characters from it
# from list of unique_chars get count for each character in each word
# build bag of characters vectors
def boc_term_vectors(word_list):
    word_list = [word.lower() for word in word_list]
    unique_chars = np.unique(
                        np.hstack([list(word) 
                                   for word in word_list]))
    word_list_term_counts = [{char: count 
                                  for char, count in np.stack(
                                      np.unique(list(word), return_counts=True), axis=1)} 
                                     for word in word_list]
    boc_vectors = [np.array([int(word_term_counts.get(char, 0)) 
                             for char in unique_chars]) 
                   for word_term_counts in word_list_term_counts]
    return list(unique_chars), boc_vectors

In [ ]:
# bag of character vectorization
import pandas as pd

feature_names, feature_vectors = boc_term_vectors(terms)
boc_df = pd.DataFrame(feature_vectors, columns=feature_names, index=terms)
print(boc_df)

In [ ]:
# store these in specific variables before computing cosine distances
root_term_boc = boc_df[vec_df.index == root_term].values[0]
other_term_bocs = [boc_df[vec_df.index == term].values[0] 
                   for term in other_terms]

* if using bag of characters based character frequencies for terms or bag of words based word frequencies for documents, the score will range from 0 to 1 because frequency vectors can never be negative; angle between two vectors cannot exceed 90 degrees
* cosine distance is complementary to the similarity score and can be computed by following formula:
$$cd(u,v) = 1-cs(u,v) = 1 - cos(\theta)$$

In [1]:
# calculate cosine distance based on formula
def cosine_distance(u,v):
    distance = 1.0 - (np.dot(u,v) / (np.sqrt(sum(np.square(u))) * np.sqrt(sum(np.square(v))))
                     )
    return distance

In [ ]:
# test similarity between example terms using bag of character representations
for term, boc_term in zip(other_terms, other_term_bocs):
    print('Analyzing similarity between root: {} and term: {}'.format(root_term, term))
    distance = round(cosine_distance(root_term_boc, boc_term), 2)
    similarity = round(1 - distance, 2)
    print('Cosine distance is {}'.format(distance))
    print('Cosine similarity is {}'.format(similarity))
    print('-'*40)